# Part 0:
## import everything
Run the cell below

In [ ]:
# -*- coding: utf-8 -*-
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import types
import string
import sys, time
import pickle
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import mlab
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
from sklearn.decomposition import KernelPCA
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter
import imageio
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False
    matplotlib.rcdefaults()
    
    root=ROOT.root
            
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'BehavioralPaper','SUPP_BadCtrl.ipynb')
    %run $currentNbPath
    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }  

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def get_ordered_colors(colormap, n):
    colors = []
    cmap = plt.cm.get_cmap(colormap)
    for colorVal in np.linspace(0, 1, n):
        colors.append(cmap(colorVal))
    return colors

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_lowercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)


In [ ]:
class TwoTailPermTest:
    """
    Permutation test as to whether there is significant difference between group one and two.
    
    group1, group2: Represent the data. they could be either one dimentional (several realizations)
        or 2-D (several realizaions through out the time/space/... course)
        EX: x.shape==(15,500) means 15 trials/samples over 500 time bins

    nIterations: Number of iterations used to shuffle. max(iterN)=(len(x)+len(y))!/len(x)!len(y)!

    initGlobConfInterval:
        Initial value for the global confidence band.

    smoothSigma: the standard deviation of the gaussian kernel used for smoothing when there are multiple data points,
        based on the Fujisawa 2008 paper, default value: 0.05

    Outputs:
        pVal: P-values
        highBand, lowBand: AKA boundary. Represents global bands.
        significantDiff: An array of True or False, indicating whether there is a difference.
    
    """  
    def __init__(self, group1, group2, nIterations=1000, initGlobConfInterval=5, smoothSigma=0.05, randomSeed=1):
        self.__group1, self.__group2 = self.__setGroupData(group1), self.__setGroupData(group2)
        self.__nIterations, self.__smoothSigma = nIterations, smoothSigma
        self.__initGlobConfInterval = initGlobConfInterval
        self.__randomSeed = randomSeed
        
        self.__checkGroups()

        # origGroupDiff is also known as D0 in the definition of permutation test.
        self.__origGroupDiff = self.__computeGroupDiff(group1, group2)

        # Generate surrogate groups, compute difference of mean for each group, and put in a matrix.
        self.__diffSurGroups = self.__setDiffSurrGroups()

        # Set statistics
        self.pVal = self.__setPVal()
        self.highBand, self.lowBand = self.__setBands()
        self.pairwiseHighBand = self.__setPairwiseHighBand()
        self.pairwiseLowBand = self.__setPairwiseLowBand()
        self.significantDiff = self.__setSignificantGroup()

    def __setGroupData(self, groupData):
        if not isinstance(groupData, dict):
            return groupData

        realizations = list(groupData.values())
        subgroups = list(groupData.keys())
                    
        dataMat = np.zeros((len(subgroups), len(realizations[0])))
        for index, realization in enumerate(realizations):
            if len(realization) != len(realizations[0]):
                raise Exception("The length of all realizations in the group dictionary must be the same")
            
            dataMat[index] = realization

        return dataMat

    def __checkGroups(self):
        # input check
        if not isinstance(self.__group1, np.ndarray) or not isinstance(self.__group2, np.ndarray):
            raise ValueError("In permutation test, \"group1\" and \"group2\" should be numpy arrays.")

        if self.__group1.ndim > 2 or self.__group2.ndim > 2:
            raise ValueError('In permutation test, the groups must be either vectors or matrices.')

        elif self.__group1.ndim == 1 or self.__group2.ndim == 1:
            self.__group1 = np.reshape(self.__group1, (len(self.__group1), 1))
            self.__group2 = np.reshape(self.__group2, (len(self.__group2), 1))

    def __computeGroupDiff(self, group1, group2):
        meanDiff = np.nanmean(group1, axis=0) - np.nanmean(group2, axis=0)
        
        if len(self.__group1[0]) == 1 and len(self.__group2[0]) == 1:
            return meanDiff
        
        return smooth(meanDiff, sigma=self.__smoothSigma)

    def __setDiffSurrGroups(self):
        # Fix seed 
        np.random.seed(seed=self.__randomSeed)
        # shuffling the data
        self.__concatenatedData = np.concatenate((self.__group1,  self.__group2), axis=0)
        
        diffSurrGroups = np.zeros((self.__nIterations, self.__group1.shape[1]))
        for iteration in range(self.__nIterations):
            # Generate surrogate groups
            # Shuffle every column.
            np.random.shuffle(self.__concatenatedData)  

            # Return surrogate groups of same size.            
            surrGroup1, surrGroup2 = self.__concatenatedData[:self.__group1.shape[0], :], self.__concatenatedData[self.__group1.shape[0]:, :]
            
            # Compute the difference between mean of surrogate groups
            surrGroupDiff = self.__computeGroupDiff(surrGroup1, surrGroup2)
            
            # Store individual differences in a matrix.
            diffSurrGroups[iteration, :] = surrGroupDiff

        return diffSurrGroups
 
    def __setPVal(self):
        positivePVals = np.sum(1*(self.__diffSurGroups > self.__origGroupDiff), axis=0) / self.__nIterations
        negativePVals = np.sum(1*(self.__diffSurGroups < self.__origGroupDiff), axis=0) / self.__nIterations
        return np.array([np.min([1, 2*pPos, 2*pNeg]) for pPos, pNeg in zip(positivePVals, negativePVals)])

    def __setBands(self):
        if not isinstance(self.__origGroupDiff, np.ndarray):  # single point comparison
            return None, None
        
        alpha = 100 # Global alpha value
        highGlobCI = self.__initGlobConfInterval  # global confidance interval
        lowGlobCI = self.__initGlobConfInterval  # global confidance interval
        while alpha >= 5:
            # highBand = np.percentile(a=self.__diffSurGroups, q=100-highGlobCI, axis=0)
            # lowBand = np.percentile(a=self.__diffSurGroups, q=lowGlobCI, axis=0)

            highBand = np.percentile(a=self.__diffSurGroups, q=100-highGlobCI)
            lowBand = np.percentile(a=self.__diffSurGroups, q=lowGlobCI)

            breaksPositive = np.sum(
                [np.sum(self.__diffSurGroups[i, :] > highBand) > 1 for i in range(self.__nIterations)]) 
            
            breaksNegative = np.sum(
                [np.sum(self.__diffSurGroups[i, :] < lowBand) > 1 for i in range(self.__nIterations)])
            
            alpha = ((breaksPositive + breaksNegative) / self.__nIterations) * 100
            highGlobCI = 0.95 * highGlobCI
            lowGlobCI = 0.95 * lowGlobCI
        return highBand, lowBand           

    def __setSignificantGroup(self):
        if not isinstance(self.__origGroupDiff, np.ndarray):  # single point comparison
            return self.pVal <= 0.05

        # finding significant bins
        globalSig = np.logical_or(self.__origGroupDiff > self.highBand, self.__origGroupDiff < self.lowBand)
        pairwiseSig = np.logical_or(self.__origGroupDiff > self.__setPairwiseHighBand(), self.__origGroupDiff < self.__setPairwiseLowBand())
        
        significantGroup = globalSig.copy()
        lastIndex = 0
        for currentIndex in range(len(pairwiseSig)):
            if (globalSig[currentIndex] == True):
                lastIndex = self.__setNeighborsToTrue(significantGroup, pairwiseSig, currentIndex, lastIndex)

        return significantGroup
    
    def __setPairwiseHighBand(self, localBandValue=0.5):        
        return np.percentile(a=self.__diffSurGroups, q=100 - localBandValue, axis=0)

    def __setPairwiseLowBand(self, localBandValue=0.5):        
        return np.percentile(a=self.__diffSurGroups, q=localBandValue, axis=0)

    def __setNeighborsToTrue(self, significantGroup, pairwiseSig, currentIndex, previousIndex):
        """
            While the neighbors of a global point pass the local band (consecutively), set the global band to true.
            Returns the last index which was set to True.
        """ 
        if (currentIndex < previousIndex):
            return previousIndex
        
        for index in range(currentIndex, previousIndex, -1):
            if (pairwiseSig[index] == True):
                significantGroup[index] = True
            else:
                break

        previousIndex = currentIndex
        for index in range(currentIndex + 1, len(significantGroup)):
            previousIndex = index
            if (pairwiseSig[index] == True):
                significantGroup[index] = True
            else:
                break
        
        return previousIndex
    
    def plotSignificant(self,ax: plt.Axes.axes,y: float,x=None,**kwargs):
        if x is None:
            x=np.arange(0,len(self.significantDiff))+1
        for x0,x1,p in zip(x[:-1],x[1:],self.significantDiff):
            if p:
                ax.plot([x0,x1],[y,y],zorder=-2,**kwargs)
                
    @staticmethod
    def plotSigPair(ax: plt.Axes.axes,y: float,x=None, s: str ='*',**kwargs):
        if x is None:
            x=(0,len(self.significantDiff))
        if 'color' not in kwargs:
            kwargs['color']='k'
        
        dy=.03*(ax.get_ylim()[1]-ax.get_ylim()[0])
        ax.plot(x,[y,y],**kwargs)
        ax.plot([x[0],x[0]],[y-dy,y],[x[1],x[1]],[y-dy,y],**kwargs)
        ax.text(np.mean(x),y,s=s,
                ha='center',va='center',color=kwargs['color'],
                size='xx-small',fontstyle='italic',backgroundcolor='w')

---

**Plot average trajectory**

In [ ]:
def plot_session_median_trajectory(data,ax,**kwargs):
    posDict=data.position
    maxL=np.nanmax(list(data.stopFrame.values()))
    maxL=int(maxL)
    position=np.ones((maxL,len(posDict.keys())))*np.nan
    time=np.arange(-data.cameraToTreadmillDelay,
                   (maxL-data.cameraSamplingRate)/data.cameraSamplingRate,
                   1/data.cameraSamplingRate)
    
    
    for i,trial in enumerate(posDict):
        pos=posDict[trial][:data.stopFrame[trial]]
        position[:len(pos),i]=pos
    
    #keeping data where 70% of points exist
    nanSum=np.sum(np.isnan(position),axis=1)
    try:
        maxTraj=np.where(nanSum>.3*position.shape[1])[0][0]
    except IndexError:
        maxTraj=position.shape[1]
    
    
    ax.plot(time[:maxTraj], np.nanmedian(position,axis=1)[:maxTraj], lw=.5, **kwargs)

    ax.set_ylabel("Position (cm)")
    ax.set_xlabel("Time (s) relative to camera start")
    

def plot_trajectory_of_lists(root,ax, profile, animalLists, colors, SessionN):
    """
    First animalList should be the main one
    """
    assert len(colors)==len(animalLists)
    Y1,Y2=param['treadmillRange']
    
    for i,animal in enumerate(animalLists[0]):
        sessions=batch_get_session_list(root, animalList=[animal], profile=profile)['Sessions']

        data=Data(root,sessions[SessionN][:6],sessions[SessionN],
                  param=param,redoPreprocess=False, saveAsPickle=False);
        data.position_correction()
        c=colors[0]
        for i,animalList in enumerate(animalLists):
            c=colors[i] if animal in animalList else c
        z=2 if c != colors[0] else -1 #first animals list animals drawn below
        plot_session_median_trajectory(data,ax,color=c,zorder=z)

    ax.set_title(f'Session #{SessionN+1}',pad=0)
    ax.set_ylim([Y1,Y2])
    ax.set_yticks([Y1,Y2/2,Y2])
    ax.set_yticklabels([Y1,'',Y2])
    ax.set_xlim([-1,8])
    ax.set_xticks([0,7])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_bounds(0,7)
    ax.set_xlabel('Trial time (s)')
    ax.set_ylabel('Position (cm)')
    ax.spines['left'].set_bounds(Y1,Y2)

In [ ]:
if "__file__" not in dir():
    profile={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':['10'],
             'Tag':'Control'
                  }
    animalList=['Rat110','Rat113','Rat120','Rat161','Rat162','Rat163','Rat164','Rat165','Rat166','Rat215',
                'Rat217','Rat218','Rat219','Rat220','Rat221','Rat222','Rat223','Rat224','Rat225','Rat226',
                'Rat227','Rat228','Rat229','Rat230','Rat231','Rat232','Rat246','Rat247','Rat248','Rat249',
                'Rat250','Rat251','Rat252','Rat253','Rat254','Rat255','Rat256','Rat257','Rat258','Rat259',
                'Rat260','Rat261','Rat262','Rat263','Rat264','Rat265','Rat297','Rat298','Rat299','Rat300',
                'Rat305','Rat306','Rat307','Rat308']
    
    weirdList=['Rat165','Rat223','Rat224','Rat228','Rat229','Rat230']
    badList=  ['Rat215','Rat218','Rat220','Rat221','Rat231','Rat256','Rat258','Rat305']
    SessionN=29
    
    animalLists=(animalList,weirdList,badList)
    colors     =([.6,.6,.6,.6],'xkcd:orange','teal')
    plt.close('all')
    fig=plt.figure(figsize=(4,4))
    ax=fig.add_subplot(111)

    
    
    plot_trajectory_of_lists(root,ax, profile, animalLists, colors, SessionN)

---

**Plot ET boxplot**

In [ ]:
def calculate_CV(root,profile,animalList,sessionRange,PERF):
    data={}
    for animal in animalList:
        allData=data_fetch(root,animal,profile,[PERF], NbSession=sessionRange)
        allData=allData.values()
        data[animal]=allData

    return data

def plot_ET_boxplot(root,ax,
                    profiles,animalLists,sessionRanges,colors,boxColors,labels,PERF="median entrance time (sec)"):
    
    widths=.3
    np.random.seed(seed=4)
    xPos=np.arange(1,len(labels)+1)
    #reading all the data
    data=[]
    for profile,animalList,sessionRange in zip (profiles,animalLists,sessionRanges):
        data.append(calculate_CV(root,profile,animalList,sessionRange,PERF))
    
    dataPool=[]
    for animalList,color,dat,x in zip(animalLists,colors,data,xPos):
        dataPool.append([])
        datapoints=list(dat.values())
        for i,(animal,pts) in enumerate(zip(animalList,datapoints)):
            vals=list(pts)[0]
            dataPool[-1].extend(vals)
            if color:
                jitter=np.random.uniform(low=x-widths/2, high=x+widths/2, size=len(vals))
                ax.scatter(jitter,vals,s=.2,c=color[i], marker='o',zorder=2)

    
    for dat,pos,c in zip(dataPool,xPos,boxColors):
        ax.boxplot(x=dat,whis=[5,95],
                      positions=[pos], widths=widths,
                      showcaps=False, showfliers=False,
                      medianprops={'color':c}, boxprops={'color':c}, whiskerprops={'color':c},
                      zorder=1)
    


    minY,maxY=ax.get_ylim()
    ax.set_ylim([minY,maxY])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_ylabel(PERF)
    ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))
    ax.spines['left'].set_bounds(ax.get_yticks()[0],ax.get_yticks()[-1])
    ax.set_xlim([xPos[0]-.5,xPos[-1]+.5])
    ax.set_xticks(xPos)
    ax.set_xticklabels(labels)
    ax.tick_params('x',labelsize='x-small',direction='in')
    for i,(ticklabel,tickline) in enumerate(zip(ax.get_xticklabels(),ax.xaxis.get_ticklines()[::2])):
        ticklabel.set_color(boxColors[i])
        tickline.set_color(boxColors[i])

    return dataPool

In [ ]:
if "__file__" not in dir():
    profile={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':'10',
             'Tag':['Control']
             }
    animalList=['Rat110','Rat113','Rat120','Rat161','Rat162','Rat163','Rat164','Rat165','Rat166','Rat215',
                'Rat217','Rat218','Rat219','Rat220','Rat221','Rat222','Rat223','Rat224','Rat225','Rat226',
                'Rat227','Rat228','Rat229','Rat230','Rat231','Rat232','Rat246','Rat247','Rat248','Rat249',
                'Rat250','Rat251','Rat252','Rat253','Rat254','Rat255','Rat256','Rat257','Rat258','Rat259',
                'Rat260','Rat261','Rat262','Rat263','Rat264','Rat265','Rat297','Rat298','Rat299','Rat300',
                'Rat305','Rat306','Rat307','Rat308']

    weirdList=['Rat165','Rat223','Rat224','Rat228','Rat229','Rat230']
    badList=  ['Rat215','Rat218','Rat220','Rat221','Rat231','Rat256','Rat258','Rat305']
    
    for animal in [*weirdList,*badList]:
        animalList.remove(animal)
    
    
    profiles=(profile,profile)
    animalLists=(animalList,badList)
    sessionRanges=(slice(20,None),)*2
    colors=([],[])
    boxColors=('k','k')
    boxColors=('k','k')
    labels=['Control','BadCtrl']
    PERF='median entrance time (sec)'
    
    plt.close('all')
    fig=plt.figure(figsize=(8,4))
    ax=fig.add_subplot(111)
    
    a=plot_ET_boxplot(root,ax,profiles,animalLists,sessionRanges,colors,boxColors,labels,PERF)

------



------

# part 2:

# **GENERATING THE FIGURE**

**Definition of Parameters**

In [ ]:
if "__file__" not in dir():
    # GRID 1 PARAMS
    #trajectory plot
    
    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'10',
             'Speed':['10'],
             'Tag':'Control'
                  }
    animalList1=['Rat110','Rat113','Rat120','Rat161','Rat162','Rat163','Rat164','Rat165','Rat166','Rat215',
                'Rat217','Rat218','Rat219','Rat220','Rat221','Rat222','Rat223','Rat224','Rat225','Rat226',
                'Rat227','Rat228','Rat229','Rat230','Rat231','Rat232','Rat246','Rat247','Rat248','Rat249',
                'Rat250','Rat251','Rat252','Rat253','Rat254','Rat255','Rat256','Rat257','Rat258','Rat259',
                'Rat260','Rat261','Rat262','Rat263','Rat264','Rat265','Rat297','Rat298','Rat299','Rat300',
                'Rat305','Rat306','Rat307','Rat308']
    
    weirdList1=['Rat165','Rat223','Rat228','Rat229','Rat230']
    badList1=  ['Rat215','Rat218','Rat220','Rat221','Rat224','Rat231','Rat256','Rat258','Rat305']
    SessionN1=29
    
    animalLists1=(animalList1,weirdList1,badList1)
    colors1     =('gray','xkcd:orange','teal')
    plt.close('all')

    
    #===============================================
    # GRID 2 PARAMS
    animalList2=list(tuple(animalList1))

    
    for animal in [*weirdList1,*badList1]:
        animalList2.remove(animal)
    
    
    profiles2=(profile1,profile1,profile1)
    animalLists2=(animalList2,weirdList1,badList1)
    sessionRanges2=(slice(20,None),)*3
    colors2=([],[],[])
    boxColors2=colors1#(colors1[0],colors1[-1])
    labels2=['Back & forth','Middle','Front']
    PERF2=['median entrance time (sec)',
           'standard deviation of entrance time',
           '% good trials']
    
    wspace2=.5
    
    
    
    #=================================================
    # GENERAL
    colorSig1='goldenrod'
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
        }
    Y1,Y2=param['treadmillRange']

**Plotting the figure**

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(7,2)
    fig=plt.figure(figsize=figsize,dpi=600)
    
    
    ###########################################
    # 1: Trajectories
    gs1= fig.add_gridspec(nrows=1, ncols=1, left=0.0, bottom=0.0, right=0.2, top=.99)
    ax1= fig.add_subplot(gs1[0])

    plot_trajectory_of_lists(root,ax1, profile1, animalLists1, colors1, SessionN1)
    ax1.text(x=7,y=10,s=f'{labels2[2]}, $n={len(animalLists1[2])}$',
             color=colors1[2],fontsize='xx-small',ha='right')
    ax1.text(x=7,y=6,s=f'{labels2[1]}, $n={len(animalLists1[1])}$',
             color=colors1[1],fontsize='xx-small',ha='right')
    ax1.text(x=7,y=2,s=f'{labels2[0]}, $n={len(animalList2)}$',
             color=colors1[0],fontsize='xx-small',ha='right')
    
    ###########################################
    #2: plot performance comparisions
    gs2= fig.add_gridspec(nrows=1, ncols=3, left=0.29, bottom=0.0, right=0.98, top=0.98,wspace=wspace2)
    ax2l= fig.add_subplot(gs2[0])
    ax2m= fig.add_subplot(gs2[1])
    ax2r= fig.add_subplot(gs2[2])
    
    d2l=plot_ET_boxplot(root,ax2l,profiles2,animalLists2,sessionRanges2,colors2,boxColors2,labels2,PERF2[0])
    d2m=plot_ET_boxplot(root,ax2m,profiles2,animalLists2,sessionRanges2,colors2,boxColors2,labels2,PERF2[1])
    d2r=plot_ET_boxplot(root,ax2r,profiles2,animalLists2,sessionRanges2,colors2,boxColors2,labels2,PERF2[2])
    
    ax2l.set_ylim([5.9,8.5])
    ax2m.set_ylim([1.1,2.75])
    ax2r.set_ylim([25,80])
    
    ax2l.spines['left'].set_bounds(6,8)
    ax2m.spines['left'].set_bounds(1.2,2.4)
    ax2r.spines['left'].set_bounds(30,70)
    
    ax2l.set_yticks([6,6.5,7,7.5,8])
    ax2m.set_yticks([1.2,1.6,2,2.4])
    ax2r.set_yticks([30,40,50,60,70])
    
    ax2l.set_ylabel('$ET$ (s)')
    ax2m.set_ylabel('$SD_{ET}$ (s)')
    ax2r.set_ylabel('% Correct trials')
    
    ax2l.tick_params('x',labelrotation=15)
    ax2m.tick_params('x',labelbottom=False,bottom=False)
    ax2r.tick_params('x',labelbottom=False,bottom=False)
    
    #PERM TESTS
    t2l=TwoTailPermTest(group1=np.array(d2l[0]), group2=np.array(d2l[1]), nIterations=10000)
    s='*' if t2l.significantDiff[0] else 'n.s.'
    t2l.plotSigPair(ax2l,y=8.2,x=(1,2), s=s, color=colorSig1,lw=.8)
    t2l_=TwoTailPermTest(group1=np.array(d2l[0]), group2=np.array(d2l[2]), nIterations=10000)
    s='*' if t2l_.significantDiff[0] else 'n.s.'
    t2l_.plotSigPair(ax2l,y=8.35,x=(1,3), s=s, color=colorSig1,lw=.8)
    
    t2m=TwoTailPermTest(group1=np.array(d2m[0]), group2=np.array(d2m[1]), nIterations=10000)
    s='*' if t2m.significantDiff[0] else 'n.s.'
    t2m.plotSigPair(ax2m,y=2.65,x=(1,2), s=s, color=colorSig1,lw=.8)
    t2m_=TwoTailPermTest(group1=np.array(d2m[0]), group2=np.array(d2m[2]), nIterations=10000)
    s='*' if t2m_.significantDiff[0] else 'n.s.'
    t2m_.plotSigPair(ax2m,y=2.74,x=(1,3), s=s, color=colorSig1,lw=.8)

    t2r=TwoTailPermTest(group1=np.array(d2r[0]), group2=np.array(d2r[1]), nIterations=10000)
    s='*' if t2r.significantDiff[0] else 'n.s.'
    t2r.plotSigPair(ax2r,y=75,x=(1,2), s=s, color=colorSig1,lw=.8)
    t2r_=TwoTailPermTest(group1=np.array(d2r[0]), group2=np.array(d2r[2]), nIterations=10000)
    s='*' if t2r_.significantDiff[0] else 'n.s.'
    t2r_.plotSigPair(ax2r,y=79,x=(1,3), s=s, color=colorSig1,lw=.8)



    
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    AXES=(ax1,ax2l,ax2m,ax2r)
    OFFX=(.06,0.05,0.05,0.05)
    OFFY=(.03,)*len(AXES)
    add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)


    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'BehavioralPaper','Figures','SUPP_BadCtrl.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()